In [1]:
import requests
import pandas as pd
import numpy as np
import plotly

In [2]:
base_url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/"
dataset_id = "hbs_exp_t121"
params = {
    'format':'JSON',
    'lang':'en',
    'startPeriod':2020,
    'endPeriod':2020    
}
#"/A.CP01+CP011+CP0111+CP01111+CP01113+CP01114+CP01116+CP01118+CP0112+CP01121+CP01122+CP01123+CP01124+CP01126_01127+CP01128A+CP01128B+CP0113+CP01131_01132+CP01133_01134+CP01135+CP01136+CP0114+CP01141+CP01142+CP01143+CP01144+CP01145+CP01146+CP01147+CP0115+CP01151+CP01152+CP01153+CP01154+CP01155+CP0116+CP0116A+CP0116B+CP0116C+CP0116D+CP0116E+CP0116F+CP0116G+CP01163+CP01164+CP0117+CP0117A+CP0117B+CP0117C+CP0117D+CP01173B+CP01173C+CP01174+CP01176+CP0118+CP01181+CP01182+CP01183+CP01184+CP01185+CP01186A+CP0119+CP01191+CP01192+CP01199+CP01199A+CP012+CP0121+CP01211+CP01212+CP01213+CP0122+CP01221+CP01222+CP01223A+CP01223C+CP02+CP021+CP0211+CP0212+CP0213+CP022+CP0220+CP023+CP0230+CP03+CP031+CP0311+CP0312+CP0313+CP0314+CP032+CP0321+CP0322+CP04+CP041+CP0411+CP0412+CP042+CP0421+CP0422+CP043+CP0431+CP0432+CP044+CP0441+CP0442+CP0443+CP0444+CP045+CP0451+CP0452+CP0453+CP0454+CP0455+CP05+CP051+CP0511+CP0512+CP0513+CP052+CP0520+CP053+CP0531+CP0532+CP0533+CP054+CP0540+CP055+CP0551+CP0552+CP056+CP0561+CP0562+CP06+CP061+CP0611+CP0612+CP0613+CP062+CP0621+CP0622+CP0623+CP063+CP0630+CP07+CP071+CP0711+CP0712+CP0713+CP0714+CP072+CP0721+CP0722+CP0723+CP0724+CP073+CP0731+CP0732+CP0733+CP0734+CP0735+CP0736+CP08+CP081+CP0810+CP082+CP0820+CP083+CP0830+CP09+CP091+CP0911+CP0912+CP0913+CP0914+CP0915+CP092+CP0921+CP0922+CP0923+CP093+CP0931+CP0932+CP0933+CP0934+CP0935+CP094+CP0941+CP0942+CP0943+CP095+CP0951+CP0952+CP0953+CP0954+CP096+CP0960+CP10+CP101+CP1010+CP102+CP1020+CP103+CP1030+CP104+CP1040+CP105+CP1050+CP11+CP111+CP1111+CP1112+CP112+CP1120+CP12+CP121+CP1211+CP1212+CP1213+CP122+CP1220+CP123+CP1231+CP1232+CP124+CP1240+CP125+CP1252+CP1253+CP1254+CP1255+CP126+CP1262+CP127+CP1270.PPS_HH.EU27_2020+EU28+EU27_2007+EU25+EU15+EA+EA18+EA17+EA13+EA12+BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE+EEA30_2007+EEA28+EFTA+NO+UK+ME+MK+RS+TR+XK/

# Construct the full URL
url = f"{base_url}{dataset_id}"

# Make the GET request to fetch data from Eurostat API
response = requests.get(url, params=params)
print(response.status_code)

200


In [3]:
data = response.json()

In [4]:
data['label']

'Mean consumption expenditure per household by COICOP consumption purpose'

In [5]:
dims = np.array(data['size'])
dim_names = data['id']

In [6]:
idx = pd.MultiIndex.from_product([pd.Series(data['dimension'][dim]['category']['index']).sort_values().index for dim in dim_names],names = dim_names)
idx_df = pd.Series(np.arange(dims.prod()),index=idx)
idx_maps = {dim:data['dimension'][dim]['category']['label'] for dim in dim_names}
dim_labels = {dim:data['dimension'][dim]['label'] for dim in dim_names}
single_dims = [dim for n, dim in zip(dims,dim_names) if n==1]
dim_labels

{'freq': 'Time frequency',
 'coicop': 'Classification of individual consumption by purpose (COICOP)',
 'unit': 'Unit of measure',
 'geo': 'Geopolitical entity (reporting)',
 'time': 'Time'}

In [7]:
def index_values(values, idx_df):
    values = pd.Series(values)
    values.index = values.index.astype(int)
    values.sort_index(inplace=True)
    values = values.reindex(idx_df.values)
    values.index = idx_df.index
    return values
values = index_values(data['value'], idx_df)
status = index_values(data['status'], idx_df)

In [8]:
country = 'BE'
country_values = values.droplevel(single_dims).xs(country,level='geo').dropna()
country_values = country_values[country_values>0]
country_data = country_values.to_frame(name='values')
country_data['label']=pd.Series(idx_maps[country_values.index.name])
country_data

,values,label
coicop,,
CP01,4913.0,Food and non-alcoholic beverages
CP011,4443.0,Food
CP0111,771.0,Bread and cereals
CP0112,1020.0,Meat
CP0113,330.0,Fish and seafood
...,...,...
CP1255,124.0,Other insurance
CP126,75.0,Financial services n.e.c.
CP1262,75.0,Other financial services n.e.c.


In [9]:
coicop = country_data.index.to_series()
tree_data = country_data.copy()

tree_data['n']   = coicop.str.len()-4
tree_data['parent']=coicop.apply(lambda x: x[0:-1] if len(x)>4 else x[0:-2])
tree_data['total'] = tree_data['values']
#for i in [1,0]:
#    tree_data.loc[tree_data['n']==i,'values'] = country_data[tree_data['n']==i+1]['values'].groupby(tree_data['parent']).sum()

for i in range(2):
    tree_data.loc[tree_data['n']==i,'values'] = np.maximum(country_data[tree_data['n']==i]['values'] - country_data[tree_data['n']==i+1]['values'].groupby(tree_data['parent']).sum(),0)

In [44]:
tree_data.loc['CP']={'n':-1,'parent':"",'label':'Total','values':0,'total':country_data.loc[tree_data['n']==0,'values'].sum()}

In [10]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'

In [45]:
import plotly.graph_objects as go

fig = go.Figure(go.Treemap(
    ids = tree_data.index.values,
    labels = tree_data['label'].values,
    values = tree_data['values'].values,
    parents = tree_data['parent'].values,
    textinfo = "label+percent entry",
    customdata = tree_data['total'].reset_index(),
    hovertemplate='<b>%{label} </b> <br><br> COICOP: %{customdata[0]} <br> PPS       : %{customdata[1]:,d}<extra></extra>',
    maxdepth=2,
    root_color='rgb(220,220,250)',
))

fig.update_layout(
    treemapcolorway = px.colors.qualitative.Pastel_r,
    margin = dict(t=50, l=25, r=25, b=25)
)
fig.show()

I have a hierarchical dataset that uses COICOP (Classification of Individual Consumption According to Purpose) codes to represent different categories and subcategories of consumption, each with a label description and values. COICOP is an international standard used to classify household consumption expenditures. It has a hierarchical structure where higher-level codes represent broader categories, and lower-level codes represent more specific subcategories. Here's a sample of the data in CSV format:

```
coicop,values,label
CP01,4913.0,Food and non-alcoholic beverages
CP011,4443.0,Food
CP0111,771.0,Bread and cereals
CP0112,1020.0,Meat
CP0113,330.0,Fish and seafood
...
CP1255,124.0,Other insurance
CP126,75.0,Financial services n.e.c.
CP1262,75.0,Other financial services n.e.c.
CP127,469.0,Other services n.e.c.
CP1270,469.0,Other services n.e.c.
```
I would like to create a treemap visualization where:

* The labels are displayed at all levels of the hierarchy instead of coicop codes.
* The top-level categories are colored, preferably using pastel colors.
* The coicop codes are displayed in the hover information at all levels (including both parent and child categories).
* Only leaf nodes should contribute to the values. Parent nodes (summary categories) should be included in the hierarchy but not contribute to the value size, as their values represent sums of their subcategories.
* In the hierarchy leave only leaf nodes as plotly will not handle it properly


In [ ]:
df = country_data.reset_index()


In [50]:
# Identify non-leaf nodes (parents of other nodes)
non_leaf = df['coicop'].apply(lambda x: any(df['coicop'].str.startswith(x) & (df['coicop'] != x)))

# Set values for non-leaf nodes to NaN to avoid their inclusion in value calculation
df['values'] = df['values'].where(~non_leaf, None)

# Create hierarchical levels based on the coicop structure
df['level1'] = df['label'].where(df['coicop'].str.len() == 4)   # Top level (e.g., CP01)
df['level2'] = df['label'].where(df['coicop'].str.len() == 5)   # Second level (e.g., CP011)
df['level3'] = df['label'].where(df['coicop'].str.len() == 6)   # Third level (e.g., CP0111)

# Fill forward the levels for only those rows where the level is missing
df['level1'] = df['level1'].fillna(method='ffill')
df['level2'] = df['level2'].fillna(method='ffill')

# Ensure there are no NaN values at any level
df = df.dropna(subset=['level1', 'level2', 'label'])

# Add `coicop` to hover data for each level (level1 and level2 nodes)
df['level1_coicop'] = df['coicop'].where(df['coicop'].str.len() == 4)
df['level2_coicop'] = df['coicop'].where(df['coicop'].str.len() == 5)
df['leaf_coicop'] = df['coicop'].where(df['coicop'].str.len() == 6)

# Fill forward coicop codes for parents
df['level1_coicop'] = df['level1_coicop'].fillna(method='ffill')
df['level2_coicop'] = df['level2_coicop'].fillna(method='ffill')

# Create a treemap using plotly with pastel colors
pastel_colors = px.colors.qualitative.Pastel  # Use Plotly's pastel color scheme

fig = px.treemap(
    df, 
    path=['level1', 'level2', 'label'],  # Use labels for hierarchy
    values='values',  # Values for the leaf nodes
    color='level1',   # Color based on the highest level (level1)
    hover_data={
        'level1_coicop': True,   # Show `coicop` for level1
        'level2_coicop': True,   # Show `coicop` for level2
        'leaf_coicop': True,     # Show `coicop` for leaf nodes
        'values': True           # Show values on hover
    },
    color_discrete_sequence=pastel_colors  # Set the color palette to pastel
)

# Show the plot
fig.show()



/tmp/ipykernel_106676/2779532372.py:13: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_106676/2779532372.py:14: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_106676/2779532372.py:25: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_106676/2779532372.py:26: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

